In [ ]:
import numpy as np

# 1) Generate ellipse points
# p(t) = R(θ) [a cos t, b sin t]^T + ε
n = 800
a = 5.0
b = 2.0
angle_deg = 25.0
noise_std = 0.25
seed = 42

rng = np.random.default_rng(seed)
t = rng.uniform(0.0, 2.0 * np.pi, size=n)

base = np.stack([a * np.cos(t), b * np.sin(t)], axis=0)  # (2, n)
angle = np.deg2rad(angle_deg)
r = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]], dtype=float)

pts = (r @ base).T  # (n, 2)
pts = pts + rng.normal(0.0, noise_std, size=pts.shape)  # ε

x = pts[:, 0]
y = pts[:, 1]


In [ ]:
import numpy as np

# 2) Covariance + eigenvectors
# μ = (1/n) Σ x_i
# z_i = x_i - μ
# Σ = (1/(n-1)) Z Z^T   where Z = [z_1 ... z_n] ∈ R^{2×n}
center_x = float(np.mean(x))
center_y = float(np.mean(y))

x0 = x - center_x
y0 = y - center_y
z = np.stack([x0, y0], axis=0)  # (2, n)

cov = (z @ z.T) / (n - 1)

# Σ v_k = λ_k v_k
# (for symmetric Σ, eigenvectors are orthonormal)
eigvals, eigvecs = np.linalg.eigh(cov)  # eigvals asc

order = np.argsort(eigvals)[::-1]
eigvals = eigvals[order]
eigvecs = eigvecs[:, order]

cov, eigvals, eigvecs


In [ ]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show

# 3) Bokeh visualization + export PNGs for README
# Var(u^T x) = u^T Σ u
# Eigenvectors maximize/minimize this variance, giving principal axes.
output_notebook()

scale = 2.0
lengths = scale * np.sqrt(np.maximum(eigvals, 0.0))
v1 = eigvecs[:, 0] * lengths[0]
v2 = eigvecs[:, 1] * lengths[1]

cx, cy = center_x, center_y

tt = np.linspace(0.0, 2.0 * np.pi, 256)
circle = np.stack([np.cos(tt), np.sin(tt)], axis=0)
curve = eigvecs @ (np.diag(lengths) @ circle)

# Step 1: point cloud
p1 = figure(width=650, height=650, match_aspect=True, title="Step 1: point cloud")
p1.circle(x, y, size=4, alpha=0.6)

# Step 2: point cloud + mean μ
p2 = figure(width=650, height=650, match_aspect=True, title="Step 2: point cloud + mean")
p2.circle(x, y, size=4, alpha=0.5)
p2.scatter([cx], [cy], marker="x", size=18, line_width=3, color="black", legend_label="mean μ")
p2.legend.location = "top_left"

# Step 3: covariance ellipse from Σ
p3 = figure(width=650, height=650, match_aspect=True, title="Step 3: ellipse implied by Σ")
p3.circle(x, y, size=4, alpha=0.15)
p3.line(curve[0, :] + cx, curve[1, :] + cy, line_width=3, color="green")

# Step 4: eigenvectors as axes
p4 = figure(width=650, height=650, match_aspect=True, title="Step 4: eigenvectors (axes)")
p4.circle(x, y, size=4, alpha=0.15)
p4.line([cx - v1[0], cx + v1[0]], [cy - v1[1], cy + v1[1]], line_width=4, color="red", legend_label="v1")
p4.line([cx - v2[0], cx + v2[0]], [cy - v2[1], cy + v2[1]], line_width=4, color="purple", legend_label="v2")
p4.legend.location = "top_left"

# Step 5: combined
p5 = figure(width=650, height=650, match_aspect=True, title="Step 5: points + ellipse + axes")
p5.circle(x, y, size=4, alpha=0.6)
p5.line(curve[0, :] + cx, curve[1, :] + cy, line_width=2, color="green", alpha=0.9, legend_label="ellipse from Σ")
p5.line([cx - v1[0], cx + v1[0]], [cy - v1[1], cy + v1[1]], line_width=3, color="red", legend_label="v1")
p5.line([cx - v2[0], cx + v2[0]], [cy - v2[1], cy + v2[1]], line_width=3, color="purple", legend_label="v2")
p5.legend.location = "top_left"

# Export to PNGs (writes into ./assets)
# NOTE: bokeh export_png requires selenium + a browser driver.
# If this fails, run:
#   python3 -m pip install selenium webdriver-manager pillow
# and make sure Chrome/Chromium is installed.
try:
    from bokeh.io import export_png

    export_png(p1, filename="assets/step1_point_cloud.png")
    export_png(p2, filename="assets/step2_mean.png")
    export_png(p3, filename="assets/step3_cov_ellipse.png")
    export_png(p4, filename="assets/step4_eigen_axes.png")
    export_png(p5, filename="assets/step5_combined.png")
    print("Saved PNGs into ./assets")
except Exception as e:
    print("PNG export failed:", repr(e))

# Show final plot inline
show(p5)
